In [95]:
import numpy as np
import altair as alt
import pandas as pd
import os
import sys
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [131]:
arquivos = [i for i in os.listdir() if 'learning_process' in i]
db = pd.DataFrame()
for arquivo in arquivos:
    _ = pd.DataFrame(np.loadtxt(arquivo)).reset_index().rename(columns = {'index':'episode',0:'t'})
    _['file'] = arquivo
    db = pd.concat([db, _])
db.head()

,episode,t,file
0,0,1.8435,learning_process___target_find_simulation__100...
1,1,15.9958,learning_process___target_find_simulation__100...
2,2,0.0546,learning_process___target_find_simulation__100...
3,3,0.2843,learning_process___target_find_simulation__100...
4,4,1.4691,learning_process___target_find_simulation__100...


In [132]:
erro = alt.Chart(db).mark_errorband(extent = 'ci', color = 'black', opacity = 0.5).encode(x = 'episode', y = 'mean(t)').properties(width = 600)
media = alt.Chart(db).mark_line(size = 1, opacity = 1, color = 'red', interpolate='basis').encode(x = 'episode', y = 'mean(t)').properties(width = 600)
erro + media

alt.LayerChart(...)

In [49]:
sys.path.insert(0, '../agents')
sys.path.insert(0, '../environments')


import ps_agent_basic_alternativo as agent_class
import env_abp_target_finding as env_class

In [50]:
env = env_class.Environment(
        100, 
        100, 
        1, 
        10000, 
        1
    )

agent = agent_class.Agent(
        env.num_actions, 
        env.num_percepts_list, 
        1, 
        1, 
        1, 
        1,
        1
    )

In [52]:
arquivos_h = [i for i in os.listdir() if 'h_matrix' in i]
arquivos_h


['h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_22-10-07.txt',
 'h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_22-50-15.txt',
 'h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_23-01-01.txt',
 'h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_23-16-56.txt',
 'h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_23-33-13.txt',
 'h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_23-44-30.txt',
 'h_matrix___target_find_simulation__100__100__1__1__standard__1__200__True__0.01__1e-05__0__10000__200000____2024-04-30_23-57-01.txt',
 'h_matrix___target_find_simulation__100__100__1

In [73]:
step_map = np.zeros(int(env.num_states*env.max_steps_per_trial))
state_map = np.zeros(int(env.num_states*env.max_steps_per_trial))
for step in range(env.max_steps_per_trial):
        for state in range(env.num_states):
            percept = agent.percept_preprocess([state,step])
            state_map[percept] = state
            step_map[percept] = step


In [129]:
db_h_matrix = pd.DataFrame()
for arquivo in arquivos:
    _ = np.loadtxt(arquivo, delimiter=',')
    _ = pd.DataFrame(_).T
    soma = _.sum(axis = 1)
    _[0] = _[0]/soma
    _[1] = _[1]/soma
    _['step'] = step_map
    _['state'] = state_map
    _
    db_h_matrix = pd.concat([db_h_matrix,_])
db_h_matrix = db_h_matrix.reset_index(drop = True)

db_h_matrix['step']  = db_h_matrix['step'].astype(int)
db_h_matrix['state']  = db_h_matrix['state'].map({0:'ABP', 1:'BP'})
db_h_matrix = db_h_matrix.rename(columns = {0:'Mantém estado', 1:'Troca estado'})

db_h_matrix = db_h_matrix.melt(id_vars = ['step','state'], value_vars=['Mantém estado','Troca estado'], var_name='Ação', value_name='Prob')
db_h_matrix = db_h_matrix.sort_values(['state','Ação','step']).reset_index(drop = True)

db_h_matrix = db_h_matrix.groupby(['state','Ação','step']).Prob.mean().reset_index()
db_h_matrix['step'] = db_h_matrix['step']/10000

db_h_matrix.head()

,state,Ação,step,Prob
0,ABP,Mantém estado,0.0000,0.996534
1,ABP,Mantém estado,0.0001,0.999408
2,ABP,Mantém estado,0.0002,0.999358
3,ABP,Mantém estado,0.0003,0.998789
4,ABP,Mantém estado,0.0004,0.999333


In [135]:
alt.Chart(db_h_matrix.query('step < 0.1')).mark_line().encode(x = 'step', y = 'Prob:Q', color = 'Ação', column = 'state', row = 'Ação').properties(width = 200, height = 100)

alt.Chart(...)